In [179]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from elasticsearch.helpers import bulk
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import csv
import pandas as pd

In [2]:
es = Elasticsearch("http://localhost:9200")

In [180]:
from elasticsearch import Elasticsearch

CLOUD_ID = 'IR_2_search:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRkZTFhZjVhMzYxYmU0M2ZlODkwYTZkNGJkYjE0YjYwMiQ4Y2U3NWNkZWMyOTU0NWYzYjBmZDM5ZWM5MDgyNTgwZg=='

ELASTIC_PASSWORD = 'EW3FiZs3r32l9yYhvwWklUVN'

es = Elasticsearch(
    cloud_id = CLOUD_ID,
    basic_auth=('elastic', ELASTIC_PASSWORD)
)

# Successful response!
es.info()
# {'name': 'instance-0000000000', 'cluster_name': ...}

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'de1af5a361be43fe890a6d4bdb14b602', 'cluster_uuid': 'mhcP4wFJTz-cmQenNPcqag', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [182]:
es.indices.create(index='words')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'words'})

In [181]:
es.indices.delete(index='words')

ObjectApiResponse({'acknowledged': True})

In [183]:
def recreate_index_stem():
    es.indices.delete(index='words')
    es.indices.create(index='words', body = settings_stem)

In [184]:
def recreate_index_nostem():
    es.indices.delete(index='words')
    es.indices.create(index='words', body = settings_nostem)

In [185]:
def check_analyzer(analyzer, text):
    body = analyzer
    body['text'] = text
    
    tokens = es.indices.analyze(index='words', body=body)['tokens']
    tokens = [token_info['token'] for token_info in tokens]
    return tokens

In [186]:
settings_stem = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'analyze':{
                        'type': 'text',
                        'analyzer': 'my_analyzer'
                    }
                }
            }
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'my_analyzer':{
                    'type': 'custom',
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                        'english_snow'
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
            'filter': {
                'english_snow':{
                    'type': 'snowball',
                    'language': 'english'
                }
            }
        }
    }
    
}

In [187]:
settings_nostem = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'analyze':{
                        'type': 'text',
                        'analyzer': 'my_analyzer'
                    }
                }
            },
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'my_analyzer':{
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
        }
    }
}

In [188]:
stemming_state = 0

In [189]:
if stemming_state == 0:
    recreate_index_nostem()
elif stemming_state == 1:
    recreate_index_stem()

C:\Users\Admin\AppData\Local\Temp\ipykernel_17400\2148422484.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='words', body = settings_nostem)


In [190]:
df = pd.read_csv('documents.csv')
data = df.to_dict('records')

In [191]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def es_actions_generator():
    for doc in data:
        yield create_es_action('words', doc['id_right'], doc)

In [192]:
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000,request_timeout = 30):
    if not ok:
        print(result)

In [193]:
def search(query, *args):
    pretty_print_result(es.search(index='words', body=query, size=20), args)
    # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20
                        
def pretty_print_result(search_result, fields=[]):
    # fields is a list of fields names which we want to be printed
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='words', id=doc_id)['_source']

In [194]:
query = {
    'query': {
        'match_all': {}
    }
}

search(query)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17400\3128239079.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='words', body=query, size=20), args)


Total documents: 10000
Doc 2085880, score is 1.0
Doc 1122599, score is 1.0
Doc 326045, score is 1.0
Doc 1441638, score is 1.0
Doc 1534400, score is 1.0
Doc 2348936, score is 1.0
Doc 606133, score is 1.0
Doc 244133, score is 1.0
Doc 222337, score is 1.0
Doc 1874211, score is 1.0
Doc 1123710, score is 1.0
Doc 1849474, score is 1.0
Doc 1422848, score is 1.0
Doc 2215766, score is 1.0
Doc 2195263, score is 1.0
Doc 588728, score is 1.0
Doc 1560615, score is 1.0
Doc 1440561, score is 1.0
Doc 2110008, score is 1.0
Doc 1193180, score is 1.0


In [195]:
text = 'the final round was held in rio de janeiro brazil the following is the brazilian roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb volleyball world league the following is the roster in the 2008 fivb'

In [196]:
analyzer = {
    'analyzer': 'my_analyzer'
}

check_analyzer(analyzer, text)


C:\Users\Admin\AppData\Local\Temp\ipykernel_17400\2247034190.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  tokens = es.indices.analyze(index='words', body=body)['tokens']


['the',
 'final',
 'round',
 'was',
 'held',
 'in',
 'rio',
 'de',
 'janeiro',
 'brazil',
 'the',
 'following',
 'is',
 'the',
 'brazilian',
 'roster',
 'in',
 'the',
 '2008',
 'fivb',
 'volleyball',
 'world',
 'league',
 'the',
 'following',
 'is',
 'the',
 'roster',
 'in',
 'the',
 '2008',
 'fivb',
 'volleyball',
 'world',
 'league',
 'the',
 'following',
 'is',
 'the',
 'roster',
 'in',
 'the',
 '2008',
 'fivb',
 'volleyball',
 'world',
 'league',
 'the',
 'following',
 'is',
 'the',
 'roster',
 'in',
 'the',
 '2008',
 'fivb',
 'volleyball',
 'world',
 'league',
 'the',
 'following',
 'is',
 'the',
 'roster',
 'in',
 'the',
 '2008',
 'fivb',
 'volleyball',
 'world',
 'league',
 'the',
 'following',
 'is',
 'the',
 'roster',
 'in',
 'the',
 '2008',
 'fivb',
 'volleyball',
 'world',
 'league',
 'the',
 'following',
 'is',
 'the',
 'roster',
 'in',
 'the',
 '2008',
 'fivb',
 'volleyball',
 'world',
 'league',
 'the',
 'following',
 'is',
 'the',
 'roster',
 'in',
 'the',
 '2008',
 'fiv

In [198]:
%%time

query = {
    'query': {
        'bool': {
            'must': {
                'match': {
                    'text_right': 'great lakes'
                }
            }
        }
    }
}

search(query)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17400\3128239079.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='words', body=query, size=20), args)


Total documents: 10000
Doc 973440, score is 15.704821
Doc 4900, score is 15.335167
Doc 1439087, score is 14.825996
Doc 1322917, score is 14.652464
Doc 2064714, score is 14.652464
Doc 1445071, score is 14.344378
Doc 617105, score is 13.750026
Doc 2334631, score is 13.677895
Doc 1217601, score is 13.677895
Doc 1374404, score is 13.677895
Doc 1037110, score is 13.481821
Doc 2326221, score is 13.323684
Doc 325041, score is 13.323684
Doc 347207, score is 13.175156
Doc 1961850, score is 12.748814
Doc 1227346, score is 12.694827
Doc 1879210, score is 12.694827
Doc 2292393, score is 12.694827
Doc 1858796, score is 12.694827
Doc 1945210, score is 12.694827
CPU times: total: 46.9 ms
Wall time: 252 ms


In [199]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L6-cos-v5')

In [200]:
import numpy as np

q = pd.read_csv('queries.csv')
runs = {}

q = q.reset_index()
runs = {}
for ind, row in q.iterrows():
    query = {
        'query': {
            'bool': {
                'must': {
                    'match': {
                        'text_right': row['text_left']
                    }
                }
            }
        }
    }
    results = es.search(index = 'words', body = query, size = 20)
    
    indcs = {}
    corpus = []

    if len(results['hits']['hits']) != 0:
        
        for i in range(0, len(results['hits']['hits'])):
            a = results['hits']['hits'][i]['_source']['text_right']
            indcs[i] = results['hits']['hits'][i]['_source']['id_right']
            corpus.append(a)

        corpus_embeddings = embedder.encode(corpus, convert_to_tensor = True)
        query_embedding = embedder.encode(row['text_left'], convert_to_tensor = True)
        
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k = 20)
        temp = {}

        for i in hits[0]:
            docid = str(indcs[i['corpus_id']])
            temp[docid] = i['score']
        runs[str(row['id_left'])] = temp

        

C:\Users\Admin\AppData\Local\Temp\ipykernel_17400\4130854387.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index = 'words', body = query, size = 20)


In [201]:
qrel = pd.read_csv('qrels')

qrels = {}
i = 0
d = {}
for j in qrel['158491\t0\t158491\t2']:
    if i != 0:
        if str(j.split()[0]) != temp:
                qrels[temp] = d
                d = {}
                temp = str(j.split()[0])
                d[str(j.split()[2])] = int(j.split()[3])

        else:
                d[str(j.split()[2])] = int(j.split()[3])

    else:
            temp = str(j.split()[0])
            i = i + 1
            d[str(j.split()[2])] = int(j.split()[3])
    qrels[temp] = d
    
qrels['158491']['158491'] = 2

In [202]:
import ir_measures
from ir_measures import *

print(ir_measures.calc_aggregate([P(rel=1)@10, P(rel=1)@20, AP], qrels, runs))

{P@20: 0.14750000000000008, P@10: 0.21299999999999994, AP: 0.16544179611007517}


In [177]:
df = pd.read_csv('BM25.res')

i = 0
d = {}
creators_runs = {}

for j in df['158491 Q0 625257 0 15.660703104969318 BM25']:
    if i != 0:
        if str(j.split()[0]) != temp:
                creators_runs[temp] = d
                d = {}
                temp = str(j.split()[0])
                d[str(j.split()[2])] = float(j.split()[4])

        else:
                d[str(j.split()[2])] = float(j.split()[4])

    else:
            temp = str(j.split()[0])
            i = i + 1
            d[str(j.split()[2])] = float(j.split()[4])
    creators_runs[temp] = d
    
creators_runs['158491']['625257'] = float(15)

In [178]:
print(ir_measures.calc_aggregate([AP, P(rel = 1)@10, P(rel = 1)@20], qrels, creators_runs))

{P@20: 0.09499999999999999, P@10: 0.1319999999999999, AP: 0.11196168401599797}
